Подготовил Семененко Денис, 21.11.15

#DMIA 2015 Fall Task 2 - Lemmatization

###Лемматизация и определение частей речи.
###https://inclass.kaggle.com/c/2015lemmatization

In [1]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

train имеет немного специфичный формат, поэтому специальная функция для считывания

In [2]:
def read_train(file_name):
    f = open(file_name)
    lines = f.readlines()
    lines_splitted = []
    for line in lines:
        try:
            splitted =  line.split(",")
            lines_splitted.append([splitted[1], splitted[2].split('+')[0], splitted[2].split('+')[1][0]])
        except: 
            "wrong lines"
    return pd.DataFrame(lines_splitted, columns=['Word', 'Init', 'Prop']) 

In [3]:
train = read_train('data/train')
test = pd.read_csv('data/test')
test['Word'] = test['X']
test.drop('X', axis=1,inplace=True)
sample_submission = pd.read_csv('data/sample_submission')

In [4]:
train[:5]

,Word,Init,Prop
0,vergognerete,vergognare,V
1,amnistiavate,amnistiare,V
2,menomazione,menomazione,N
3,sfaldavamo,sfaldare,V
4,sfodererei,sfoderare,V


#Cross validation

In [5]:
xtrain, xcv = train_test_split(train, test_size = 0.2, random_state = 10)

Попробуем раздельно предсказывать часть речи и начальную форму. В качестве признаков будем использовать n-грамы.

In [6]:
cv = CountVectorizer(analyzer='char_wb', ngram_range=(2, 3), min_df=1)
ngram_xtrain = cv.fit_transform(xtrain['Word'])
ngram_xcv = cv.transform(xcv['Word'])
lr = LogisticRegression()
lr.fit(ngram_xtrain, xtrain['Prop'])
pred = lr.predict(ngram_xcv)
accuracy_score(xcv['Prop'], pred)

0.92780681051921776

Теперь надо придумать, каким образом предсказывать начальную форму. 


Я буду решать эту задачу, как задачу классификации, где класс это пара: количество отсеченных букв и добавленное окончане.

Для этого мне придется посчитать две эти величины, буду использовать для этого следующие функции:

In [7]:
#считает длину общего префикса двух слов
def lenCommonPrefix(s1, s2):
    i = 0
    while i < len(s1) and i < len(s2) and s1[i] == s2[i]:
        i += 1
    return i
#считает отсечение
def calcCut(s):
    return len(s['Word']) - lenCommonPrefix(s['Word'], s['Init'])
#получает окончание
def getEnd(s):
    init = s['Init']
    lenCommon = len(s['Word']) - s['Cut']
    lenEnd = len(init) - lenCommon
    if lenEnd != 0:
        return init[-lenEnd:]
    return '_'
#объединяет две величины в один столбец, который и будет классом
def codeCutEnd(s):
    return str(s['Cut']) + "+" + s['End']

Сделаем все наши пребразования:

In [8]:
xtrain['Cut'] = xtrain.apply(calcCut, axis=1)
xtrain['End'] = xtrain.apply(getEnd, axis=1)
xtrain['Label'] = xtrain.apply(codeCutEnd, axis=1)
xtrain[:5]

/home/denny_sem/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/denny_sem/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
/home/denny_sem/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

,Word,Init,Prop,Cut,End,Label
37473,affacceremmo,affacciare,V,6,iare,6+iare
96838,manifatturo,manifatturare,V,1,are,1+are
64157,raccapricciate,raccapricciare,V,2,re,2+re
99999,inveleni,invelenire,V,0,re,0+re
7048,serpei,serpere,V,1,re,1+re


И над валидационной выборкой:

In [9]:
xcv['Cut'] = xcv.apply(calcCut, axis=1)
xcv['End'] = xcv.apply(getEnd, axis=1)
xcv['Label'] = xcv.apply(codeCutEnd, axis=1)
xcv[:5]

/home/denny_sem/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/denny_sem/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
/home/denny_sem/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

,Word,Init,Prop,Cut,End,Label
47004,aggiuntato,aggiuntare,V,2,re,2+re
58581,primeggi,primeggiare,V,0,are,0+are
79287,serbatoio,serbatoio,N,0,_,0+_
97863,conservatorio,conservatorio,N,0,_,0+_
67460,desecretan,desecretare,V,1,re,1+re


Посмотрим, какое количество классов у нас получилось:

In [10]:
len(set(xtrain['Label']))

224

Используя в качестве признаков n-грамы, делаем предсказание

In [11]:
lr = LogisticRegression()
lr.fit(ngram_xtrain, xtrain['Label'])
pred = lr.predict(ngram_xcv)
accuracy_score(xcv['Label'], pred)

0.88170094403236687

###Делаем сабмит

Считаем наши величины на всей тренировочной выборке.

In [12]:
train['Cut'] = train.apply(calcCut, axis=1)
train['End'] = train.apply(getEnd, axis=1)
train['Label'] = train.apply(codeCutEnd, axis=1)

In [13]:
cv = CountVectorizer(analyzer='char_wb', ngram_range=(2,3), min_df=1)
ngram_train = cv.fit_transform(train['Word'])
ngram_test = cv.transform(test['Word'])

Предсказываем часть речи

In [14]:
lr = LogisticRegression()
lr.fit(ngram_train, train['Prop'])
test['Prop'] = lr.predict(ngram_test) 

Предсказываем начальную форму

In [15]:
lr = LogisticRegression()
lr.fit(ngram_train, train['Label'])
test['Label'] = lr.predict(ngram_test)

In [16]:
def transform(s):
    cut = int(s['Label'].split('+')[0])
    end = s['Label'].split('+')[1]
    word = s['Word']
    prop = s['Prop']
    init = word[:-cut] + end
    init.replace('-','')#case of empty endian
    return init + '+' + prop

test['Category'] = test.apply(transform, axis=1)

In [17]:
submit = test[['Id', 'Category']]
submit[:5]

,Id,Category
0,1,gettonare+V
1,2,incidentale+A
2,3,involtare+V
3,4,liere+V
4,5,comunistizzare+V


In [18]:
submit.to_csv('benchmark.csv', index=False)